In [55]:
import numpy as np
import pandas as pd

from sklearn import preprocessing #if it says that this dosent work look at the python 3 in the top right.

In [56]:
ORIGINAL_TRAINING_FILE = '/Users/ethan/Desktop/Ethan/Python/ML/framework/input/spaceship-titanic/train.csv'
ORIGINAL_TEST_FILE = '/Users/ethan/Desktop/Ethan/Python/ML/framework/input/spaceship-titanic/test.csv'



In [63]:
df = pd.read_csv(ORIGINAL_TRAINING_FILE)
test_df = pd.read_csv(ORIGINAL_TEST_FILE)

# df.corr() this shows us that age, food court and shoppingmall pretty much dont matter:
#I will confirm this using the feature importances thing but still I belive they are unimportant
#for the categorical values I will add another value for if it is null -  this can be handelled in the encode func.
#df.isna().sum()

#now for the numerical values we only need to worry about the room service, spa and vrdeck:
#df.drop(categorical_values + values_to_be_dropped, axis=1).isna().sum()
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [64]:
cabin_prefix = []
for string in df['Cabin'].values:
    if isinstance(string, float):
        cabin_prefix.append(string)
    else: 
        cabin_prefix.append(string.split('/')[0])

In [65]:
df['CabinPrefix']=cabin_prefix

df['RoomService'] = df['RoomService'].fillna(df['RoomService'].mean())
df['Spa'] = df['Spa'].fillna(df['Spa'].mean())
df['VRDeck'] = df['VRDeck'].fillna(df['VRDeck'].mean())


#repeating for the test data:
#test_df['hasCabin']=test_df['Cabin'].apply(lambda x: 0 if x==0 else 1)
#test_df['Family'] = test_df['SibSp'] + test_df['Parch']
#test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
#test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())


In [38]:
df

PassengerId      0
RoomService      0
Spa              0
VRDeck           0
Transported      0
CabinPrefix    199
dtype: int64

In [66]:
values_to_be_dropped = ['FoodCourt', 'Name', 'ShoppingMall', 'Age', 'Cabin']
categorical_values = ['VIP', 'Destination', 'CryoSleep', 'HomePlanet', 'CabinPrefix']

#test_df = test_df.drop(values_to_be_dropped, axis = 1)
df = df.drop(values_to_be_dropped, axis = 1)

In [67]:
#using the one hot encoder:

def encode_categories(cat_values, dataframe):
    ohe = preprocessing.OneHotEncoder(sparse=False)
    for cat in cat_values:
        ohe_category = ohe.fit_transform(dataframe[cat].values.reshape(-1, 1))
        for i in range(ohe_category.shape[1]):
            title = cat + f'_{i}'
            dataframe[title] = ohe_category[:, i] #this is all of the values in the i column of the table
    dataframe = dataframe.drop(cat_values, axis = 1)
    return dataframe
            
df = encode_categories(categorical_values, df)
#test_df = encode_categories(categorical_values, test_df)

In [69]:
df.isna().sum()

PassengerId      0
RoomService      0
Spa              0
VRDeck           0
Transported      0
VIP_0            0
VIP_1            0
VIP_2            0
Destination_0    0
Destination_1    0
Destination_2    0
Destination_3    0
CryoSleep_0      0
CryoSleep_1      0
CryoSleep_2      0
HomePlanet_0     0
HomePlanet_1     0
HomePlanet_2     0
HomePlanet_3     0
CabinPrefix_0    0
CabinPrefix_1    0
CabinPrefix_2    0
CabinPrefix_3    0
CabinPrefix_4    0
CabinPrefix_5    0
CabinPrefix_6    0
CabinPrefix_7    0
CabinPrefix_8    0
dtype: int64

In [70]:
#repalce the train data with this:
df.to_csv('/Users/ethan/Desktop/Ethan/Python/ML/framework/notebooks/processed_train.csv', index=False)
#test_df.to_csv('/Users/ethan/Desktop/Ethan/Python/ML/framework/notebooks/processed_test.csv', index=False)